In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)

    val_labels = sampler.test_data[sampler.test_mask]

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None

    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )

  0%|          | 0/823 [00:00<?, ?it/s]

epoch:   0 loss:0.701953 auc:0.4696
epoch:  20 loss:0.352371 auc:0.9460
epoch:  40 loss:0.328265 auc:0.9436
epoch:  60 loss:0.307123 auc:0.9440
epoch:  80 loss:0.281943 auc:0.9474
epoch: 100 loss:0.262346 auc:0.9500
epoch: 120 loss:0.250724 auc:0.9505
epoch: 140 loss:0.242036 auc:0.9480
epoch: 160 loss:0.236298 auc:0.9483
epoch: 180 loss:0.234846 auc:0.9491
epoch: 200 loss:0.229564 auc:0.9456
epoch: 220 loss:0.228158 auc:0.9479
epoch: 240 loss:0.225704 auc:0.9426
epoch: 260 loss:0.228235 auc:0.9491
epoch: 280 loss:0.223018 auc:0.9423
epoch: 300 loss:0.223282 auc:0.9424
epoch: 320 loss:0.221113 auc:0.9406
epoch: 340 loss:0.220554 auc:0.9321
epoch: 360 loss:0.219219 auc:0.9401
epoch: 380 loss:0.220255 auc:0.9282
epoch: 400 loss:0.217850 auc:0.9334
epoch: 420 loss:0.221835 auc:0.9363
epoch: 440 loss:0.217344 auc:0.9389
epoch: 460 loss:0.216738 auc:0.9396
epoch: 480 loss:0.215976 auc:0.9304
epoch: 500 loss:0.217776 auc:0.9254
epoch: 520 loss:0.215449 auc:0.9235
epoch: 540 loss:0.215894 auc

  0%|          | 1/823 [01:02<14:18:16, 62.65s/it]

Fit finished.
epoch:   0 loss:0.702031 auc:0.4609
epoch:  20 loss:0.351768 auc:0.9175
epoch:  40 loss:0.327758 auc:0.9223
epoch:  60 loss:0.303871 auc:0.9230
epoch:  80 loss:0.280287 auc:0.9266
epoch: 100 loss:0.261794 auc:0.9289
epoch: 120 loss:0.259215 auc:0.9181
epoch: 140 loss:0.242867 auc:0.9280
epoch: 160 loss:0.236359 auc:0.9253
epoch: 180 loss:0.232969 auc:0.9244
epoch: 200 loss:0.229601 auc:0.9264
epoch: 220 loss:0.227233 auc:0.9258
epoch: 240 loss:0.228185 auc:0.9198
epoch: 260 loss:0.223939 auc:0.9234
epoch: 280 loss:0.227743 auc:0.9314
epoch: 300 loss:0.221947 auc:0.9207
epoch: 320 loss:0.223350 auc:0.8925
epoch: 340 loss:0.219900 auc:0.9191
epoch: 360 loss:0.223878 auc:0.9058
epoch: 380 loss:0.218534 auc:0.9108
epoch: 400 loss:0.220089 auc:0.9325
epoch: 420 loss:0.218273 auc:0.9165
epoch: 440 loss:0.216461 auc:0.9162
epoch: 460 loss:0.218415 auc:0.9097
epoch: 480 loss:0.215843 auc:0.9194
epoch: 500 loss:0.217068 auc:0.9016
epoch: 520 loss:0.215040 auc:0.9102
epoch: 540 los

  0%|          | 2/823 [02:03<14:00:12, 61.40s/it]

...

Fit finished.
epoch:   0 loss:0.698134 auc:0.4909
epoch:  20 loss:0.354314 auc:0.9425
epoch:  40 loss:0.329743 auc:0.9485
epoch:  60 loss:0.306808 auc:0.9444
epoch:  80 loss:0.283092 auc:0.9403
epoch: 100 loss:0.263393 auc:0.9370
epoch: 120 loss:0.253215 auc:0.9396
epoch: 140 loss:0.242291 auc:0.9383
epoch: 160 loss:0.237246 auc:0.9359
epoch: 180 loss:0.232867 auc:0.9370
epoch: 200 loss:0.230107 auc:0.9356
epoch: 220 loss:0.228670 auc:0.9308
epoch: 240 loss:0.225910 auc:0.9336
epoch: 260 loss:0.224917 auc:0.9247
epoch: 280 loss:0.226068 auc:0.9315
epoch: 300 loss:0.222070 auc:0.9351
epoch: 320 loss:0.223433 auc:0.9284
epoch: 340 loss:0.220353 auc:0.9362
epoch: 360 loss:0.221528 auc:0.9292
epoch: 380 loss:0.218862 auc:0.9396
epoch: 400 loss:0.221144 auc:0.9394
epoch: 420 loss:0.217832 auc:0.9352
epoch: 440 loss:0.228929 auc:0.9193
epoch: 460 loss:0.218459 auc:0.9415
epoch: 480 loss:0.216634 auc:0.9454
epoch: 500 loss:0.216147 auc:0.9402
epoch: 520 loss:0.216303 auc:0.9398
epoch: 540 los

  3%|▎         | 25/823 [25:00<13:15:59, 59.85s/it]

Fit finished.
epoch:   0 loss:0.699578 auc:0.4887
epoch:  20 loss:0.353999 auc:0.9371
epoch:  40 loss:0.328989 auc:0.9386
epoch:  60 loss:0.306998 auc:0.9398
epoch:  80 loss:0.282597 auc:0.9394
epoch: 100 loss:0.263479 auc:0.9367
epoch: 120 loss:0.250503 auc:0.9340
epoch: 140 loss:0.243551 auc:0.9351
epoch: 160 loss:0.236456 auc:0.9330
epoch: 180 loss:0.232459 auc:0.9306
epoch: 200 loss:0.229559 auc:0.9319
epoch: 220 loss:0.227808 auc:0.9303
epoch: 240 loss:0.228860 auc:0.9312
epoch: 260 loss:0.224144 auc:0.9309
epoch: 280 loss:0.222549 auc:0.9320
epoch: 300 loss:0.222693 auc:0.9281
epoch: 320 loss:0.220598 auc:0.9309
epoch: 340 loss:0.221807 auc:0.9290
epoch: 360 loss:0.219196 auc:0.9306
epoch: 380 loss:0.221068 auc:0.9297
epoch: 400 loss:0.218001 auc:0.9298
epoch: 420 loss:0.218698 auc:0.9294
epoch: 440 loss:0.216877 auc:0.9298
epoch: 460 loss:0.218192 auc:0.9240
epoch: 480 loss:0.216084 auc:0.9284
epoch: 500 loss:0.225593 auc:0.9148
epoch: 520 loss:0.216438 auc:0.9237
epoch: 540 los

  3%|▎         | 26/823 [26:00<13:16:24, 59.96s/it]

Fit finished.
epoch:   0 loss:0.700736 auc:0.5221
epoch:  20 loss:0.353725 auc:0.8837
epoch:  40 loss:0.329478 auc:0.8948
epoch:  60 loss:0.306651 auc:0.8966
epoch:  80 loss:0.282928 auc:0.8950
epoch: 100 loss:0.265337 auc:0.8971
epoch: 120 loss:0.250824 auc:0.8923
epoch: 140 loss:0.242873 auc:0.8879
epoch: 160 loss:0.237114 auc:0.8881
epoch: 180 loss:0.233282 auc:0.8935
epoch: 200 loss:0.230053 auc:0.8942
epoch: 220 loss:0.227689 auc:0.8913
epoch: 240 loss:0.225166 auc:0.8921
epoch: 260 loss:0.226127 auc:0.8995
epoch: 280 loss:0.222592 auc:0.8926
epoch: 300 loss:0.223349 auc:0.9000
epoch: 320 loss:0.220511 auc:0.8961
epoch: 340 loss:0.220823 auc:0.8927
epoch: 360 loss:0.219189 auc:0.8935
epoch: 380 loss:0.219332 auc:0.9030
epoch: 400 loss:0.218294 auc:0.8971
epoch: 420 loss:0.218440 auc:0.9004
epoch: 440 loss:0.217244 auc:0.9001
epoch: 460 loss:0.216963 auc:0.8983
epoch: 480 loss:0.221950 auc:0.8969
epoch: 500 loss:0.215944 auc:0.8969
epoch: 520 loss:0.215843 auc:0.8973
epoch: 540 los

In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")